In [1]:
import numpy as np
import pandas as pd

from src.book_collaborative_filtering.evaluator import Evaluator

In [ ]:
number_of_runs = 100
ratings = pd.read_csv("ratings.csv")

# Baseline

In [3]:
params = {
    "neighborhood_method": None,
    "correlation_method": "pearson",
    "minimal_similarity": -1,
    "number_of_neighbors": 0,
    "minimum_number_of_books_rated_in_common": 1,
    "minimal_number_of_ratings": 1,
    "number_of_runs": 10,
    "deviation_from_mean": False,
}

evaluator = Evaluator(ratings=ratings, params=params)
evaluator.run_parallel(number_of_runs=number_of_runs)

metrics = pd.DataFrame(evaluator.metrics)
metrics = metrics.join(pd.DataFrame(params, index=[0]), how='cross')
metrics.head()
metrics.to_csv("results/book_cf_results.csv")